In [1]:
def load_rules(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        rules_text = file.read()
    return rules_text

def preprocess_document(document):
    # Split document into lines
    chunks = document.split('\n')
    # Remove any empty lines
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    return chunks

# Load and preprocess the document
document = load_rules("ultimate_frisbee_rules-manual_copy_from_website-edited.txt")
chunks = preprocess_document(document)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Vectorize the chunks
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(chunks)

# Store vectors and chunks
index = {i: chunk for i, chunk in enumerate(chunks)}

In [7]:
def retrieve_relevant_chunks(query, vectorizer, X, index, top_n=5):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, X).flatten()
    relevant_indices = np.argsort(similarities, axis=0)[-top_n:][::-1]
    return [index[i] for i in relevant_indices]

query = "Explain the timeout rules"
relevant_chunks = retrieve_relevant_chunks(query, vectorizer, X, index)
print(relevant_chunks)

['2.D.5. explain their viewpoint clearly and concisely;', '7.E.3. After the spirit timeout:', '20.E. If a novice player commits an infraction out of sincere ignorance of the rules, it should be common practice to stop play and explain the infraction.', '2.H. In the case where a novice player commits an infraction out of ignorance of the rules, experienced players are obliged to explain the infraction and clarify what should happen.', '15.A.5.b. Specific Rules:']


In [ ]:
def load_rules(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        rules_text = file.read()
    return rules_text

def preprocess_document(document):
    # Split document into lines
    chunks = document.split('\n')
    # Remove any empty lines
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    return chunks

# Load and preprocess the document
document = load_rules("ultimate_frisbee_rules-manual_copy_from_website-edited.txt")
chunks = preprocess_document(document)

In [16]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Vectorize the chunks
chunk_embeddings = model.encode(chunks, convert_to_tensor=True)

# Store vectors and chunks
index = {i: chunk for i, chunk in enumerate(chunks)}

def retrieve_relevant_chunks(query, model, chunk_embeddings, index, top_n=5):
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, chunk_embeddings)[0]
    similarities = similarities.cpu().numpy()  # Move to CPU and convert to NumPy array
    relevant_indices = np.argsort(similarities)[-top_n:][::-1]
    return [index[i] for i in relevant_indices]

# query = "Explain the timeout rules"
query = "What is the stall count?"
relevant_chunks = retrieve_relevant_chunks(query, model, chunk_embeddings, index)
print(relevant_chunks)

['15.A.1. The stall count consists of announcing “stalling” and counting from one to ten loudly enough for the thrower to hear.', '15.A.5. If a stall count is interrupted by a call, the thrower and marker are responsible for agreeing on the correct count before the check. The count reached is the last number fully uttered by the marker before the call. The count is resumed with the word “stalling” followed by the number listed below:', '7.D.4.a.2. If the technical timeout stopped play, the count resumes at the stall count reached plus one, or at six if over five.', '15.A.2.b. However, unless 15.A.2.a applies, the stall count may not be initiated or resumed before a pivot is established:', '15.B.6.c. If this (15.B.6.b) occurs in the same possession following a contested stall (either due to 15.B.6.a or 15.A.3.b), the stall count resumes at six.']


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

def generate_response(query, relevant_chunks, model, tokenizer):
    # Combine the relevant chunks into a single context
    context = " ".join(relevant_chunks)
    input_text = f"Query: {query}\nContext: {context}\nAnswer:"
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(inputs, max_length=500, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example query
query = "Explain the timeout rules"
relevant_chunks = retrieve_relevant_chunks(query, model, chunk_embeddings, index)
response = generate_response(query, relevant_chunks, model, tokenizer)
print(response)